# Optische Bilder / Pan-STARRs


Das `panstarrs.py`-Modul dient zum Bezug der optischen Bilddateien des Pan-STARRs Datenreleases. Hierzu werden wie bereits in Notebook 01 beschrieben, die RA/DEC Koordinaten der Objekte, die
sich im Shimwellkatalog befinden genutzt. Die Objektausschnitte werden hierbei jedoch nicht auf Basis der Mosaic-Dateien des LoTSS-Data Release 1 erzeugt sondern hierzu wird
die API von Pan-STARRS genutzt. Mithilfe folgender Funktionsaufrufe lässt sich aus einem gegebenen Katalog ein PINK-File mit Bildausschnitten aus dem optischen Spektrum erzeugen.


In [1]:
# Zum Einlesen des Shimwellkatalogs als Pandas-DataFrame
from hda_fits import fits as hfits

# Zum Erzeugen und Auslesen von PINK-Dateien
from hda_fits import pink as hpink

# Zum Beziehen der Pan-STARRS Daten
from hda_fits import panstarrs as hpan

import pandas as pd

Wir wollen nun ein optisches PINK-File erzeugen, dass Cross-gematchte Bildausschnitte zu den in Notebook 01 erzeugten Objekten im Radiospektrum enthält.
Wir laden deshalb zunächst den gespeicherten Katalog, den wir bei der erzeugung des Radiobilder-PINK-Files ebenfalls unter `data/` gespeichert haben.


In [2]:
cat = pd.read_parquet("data/my_written_catalog_radio.parquet")
cat

,Source_Name,RA,E_RA,E_RA_tot,DEC,E_DEC,E_DEC_tot,Peak_flux,E_Peak_flux,E_Peak_flux_tot,...,DC_Min,E_DC_Min,PA,E_PA,DC_PA,E_DC_PA,Isl_rms,S_Code,Mosaic_ID,Isl_id
4054,ILTJ120800.39+495607.9,182.001646,0.013954,0.124349,49.935548,0.014009,0.124355,19.182606,0.101341,3.837859,...,1.638332,0.032797,149.722174,54.109835,149.722174,54.109835,0.099546,C,P22Hetdex04,5006
7621,ILTJ111100.50+521444.3,167.752075,0.021597,0.174373,52.245640,0.015568,0.173730,17.045323,0.102154,3.410595,...,2.662402,0.036482,95.356292,1.018058,95.356292,1.018058,0.096193,C,P8Hetdex,3935
7622,ILTJ111058.20+521444.3,167.742506,0.063235,0.184223,52.245660,0.044029,0.178545,6.260779,0.102967,1.256382,...,3.219402,0.103381,93.733257,3.184735,93.733257,3.184735,0.096193,C,P8Hetdex,3935
18022,ILTJ121215.29+533054.9,183.063689,0.049877,0.134335,53.515269,0.030321,0.128365,7.722341,0.087457,1.546942,...,3.490728,0.070651,84.203569,2.229824,84.203569,2.229824,0.081719,C,P182+55,2592
18023,ILTJ121213.83+533055.7,183.057642,0.049263,0.134109,53.515487,0.035989,0.129821,7.160457,0.087946,1.434789,...,3.828152,0.080531,107.892602,2.909930,107.892602,2.909930,0.081719,C,P182+55,2592


# Optische (Roh-)Daten von Pan-STARRS API beziehen

Mithilfe von `get_images_panstarrs` können die rohen Bildausschnitte an den Koordinaten der Objekte des Objektkatalogs bezogen werden. Es werden hierbei für jedes Objekt mehrere Channels in unterschiedlichen
Farbspektren bezogen.

In [3]:
hpan.get_images_panstarrs(cat, "data/panstarrs/", size=500)

2022-02-05 10:23:31,958 panstarrs.py: 29.7 s: loaded 15 FITS files from 15 expected FITS files at different bands


## Erzeugung des optischen PINK-Files und Katalog

Nun kann das optische PINK-File erzeugt werden, dass in Kombination mit dem bereits in Notebook 01 erzeugten PINK-File im Radiospektrum in Notebook 03 zu einem Multichannel-PINK-File kombiniert wird.
Es wird außerdem ein Katalog erzeugt und gespeichert, der die Untermenge der Objekte des ursprünglichen Radiokatalogs enthält, die tatsächlich mithilfe von PanSTARRS gecrossmatched werden konnte ie. für
die tatsächlich optische Ausschnitte bezogen werden konnten.

In [7]:
cat_optical = hpink.write_panstarrs_objects_to_pink_file(
    filepath="data/my_pink_file_optical.pink",
    panstarrs_catalog=cat, 
    panstarrs_data_path="data/panstarrs/", 
    download=True, 
    image_size=95)

2022-02-05 10:28:56,318 panstarrs.py: 15.5 s: loaded 3 FITS files from 3 expected FITS files at different bands
2022-02-05 10:28:56,320 panstarrs.py: Loading data/panstarrs/ILTJ120800.39+495607.9_filter=r.fits
2022-02-05 10:28:56,333 panstarrs.py: Loading data/panstarrs/ILTJ120800.39+495607.9_filter=g.fits
2022-02-05 10:28:56,342 panstarrs.py: Loading data/panstarrs/ILTJ120800.39+495607.9_filter=i.fits
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
2022-02-05 10:28:56,408 logger.py: FITSFixedWarning: PC001001= -1. 
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
2022-02-05 10:28:56,423 logger.py: FITSFixedWarning: PC001002= 0. 
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
2022-02-05 10:28:56,438 logger.py: FITSFixedWarning: PC002001= 0. 
this form of the PCi_ja keyword is deprecat

In [8]:
cat_optical

,Source_Name,RA,E_RA,E_RA_tot,DEC,E_DEC,E_DEC_tot,Peak_flux,E_Peak_flux,E_Peak_flux_tot,...,DC_Min,E_DC_Min,PA,E_PA,DC_PA,E_DC_PA,Isl_rms,S_Code,Mosaic_ID,Isl_id
4054,ILTJ120800.39+495607.9,182.001646,0.013954,0.124349,49.935548,0.014009,0.124355,19.182606,0.101341,3.837859,...,1.638332,0.032797,149.722174,54.109835,149.722174,54.109835,0.099546,C,P22Hetdex04,5006
7621,ILTJ111100.50+521444.3,167.752075,0.021597,0.174373,52.245640,0.015568,0.173730,17.045323,0.102154,3.410595,...,2.662402,0.036482,95.356292,1.018058,95.356292,1.018058,0.096193,C,P8Hetdex,3935
7622,ILTJ111058.20+521444.3,167.742506,0.063235,0.184223,52.245660,0.044029,0.178545,6.260779,0.102967,1.256382,...,3.219402,0.103381,93.733257,3.184735,93.733257,3.184735,0.096193,C,P8Hetdex,3935
18022,ILTJ121215.29+533054.9,183.063689,0.049877,0.134335,53.515269,0.030321,0.128365,7.722341,0.087457,1.546942,...,3.490728,0.070651,84.203569,2.229824,84.203569,2.229824,0.081719,C,P182+55,2592
18023,ILTJ121213.83+533055.7,183.057642,0.049263,0.134109,53.515487,0.035989,0.129821,7.160457,0.087946,1.434789,...,3.828152,0.080531,107.892602,2.909930,107.892602,2.909930,0.081719,C,P182+55,2592
